In [11]:
import torch
import sys
sys

mask = torch.Tensor([0, 1, 0]).reshape(1, -1)
psi = torch.ones((3, 2))


In [12]:
print(mask.shape, psi.shape)
print(mask * psi)

torch.Size([1, 3]) torch.Size([3, 2])


RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1

In [25]:
import numpy as np
psi = np.ones((20, 10))
mask = np.eye(20)[1]

shape = psi.shape[1:]

for i in range(len(psi)):
    if mask[i] == 0:
        psi[i, :] = np.zeros(shape)
        
print(psi)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [2]:
import d4rl
import gym

env = gym.make('hopper-expert-v0')
dataset = env.get_dataset()

/home/yiranwang/anaconda3/envs/DPC/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [12]:
import sys 
sys.path.append('../')
from deep_rl import *

actions = dataset['actions']
observations = dataset['observations']
terminals = dataset['terminals']
rewards = dataset['rewards']

replay = UniformReplay(memory_size=int(1e6), batch_size=256)

for i in range(len(rewards) - 1):
    # if the next state is still within the same episode
    if not terminals[i]:
        obs = observations[i].reshape(1, -1)
        action = actions[i].reshape(1, -1)
        next_obs = observations[i].reshape(1, -1)
        reward = rewards[i].reshape(1, -1)
        done = terminals[i+1].reshape(1, -1)
        replay.feed(dict(
            state=obs,
            action=action,
            reward=reward,
            next_state=next_obs,
            mask=1-np.asarray(done, dtype=np.int32),
        ))

In [89]:
transitions = replay.sample()
states = tensor(transitions.state)
actions = tensor(transitions.action)
rewards = tensor(transitions.reward).unsqueeze(-1)
next_states = tensor(transitions.next_state)
mask = tensor(transitions.mask).unsqueeze(-1)

In [16]:
print(mask.reshape(-1).shape)

torch.Size([256])


In [45]:
states.shape
m = torch.zeros([256, 11])

In [43]:
actions.shape

torch.Size([256, 3])

In [90]:
mask[3]

tensor([[1.]])

In [93]:
mask[3] = torch.zeros(mask[0].shape)
mask = mask.reshape(mask.shape[0], -1)
print(mask.shape)
print(mask.reshape(mask.shape[0], -1).expand(256, 11).shape)
mask = mask.reshape(mask.shape[0], -1).expand(256, 11)
print(mask[3])
# mask.repeat(mask.shape[0], 11)

RuntimeError: unsupported operation: more than one element of the written-to tensor refers to a single memory location. Please clone() the tensor before performing the operation.

In [96]:
print(mask[2])

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
